In [1]:
from requests import get
from requests.auth import HTTPBasicAuth
from IPython.core.interactiveshell import InteractiveShell
from pandas import DataFrame

from core.settings import setting



InteractiveShell.ast_node_interactivity = "all"

In [2]:
oauth = HTTPBasicAuth(setting.email_comcare,setting.password_comcare)

In [3]:
pay = {'xmlns':setting.xmlns, 'limit':setting.limit}
forms = get(setting.form_base_url,auth=oauth,params=pay).json()
__objects = forms["objects"]
__meta = forms["meta"]
print(__meta)

while __meta["next"]:
    __forms = get(f"""{setting.form_base_url}{__meta['next']}""",auth=oauth).json()
    __objects += __forms["objects"]
    __meta = __forms["meta"]

len(__objects)

{'limit': 1000, 'next': '?xmlns=http%3A%2F%2Fopenrosa.org%2Fformdesigner%2FDD8FE78F-C33C-4D7D-A63B-AEDFF3A85279&limit=1000&offset=1000', 'offset': 0, 'previous': None, 'total_count': 1657}


1657

In [4]:
pay_cases = {'type':setting.types,'limit':setting.caselimit}
form_cases = get(setting.case_base_url,auth=oauth,params=pay_cases).json()
__objects_cases = form_cases["objects"]
__meta_cases = form_cases["meta"]
print(__meta_cases)

while __meta_cases["next"]:
    __form_cases = get(f"""{setting.case_base_url}{__meta_cases['next']}""",auth=oauth).json()
    __objects_cases += __form_cases["objects"]
    __meta_cases = __form_cases["meta"]

len(__objects_cases)


{'limit': 5000, 'next': None, 'offset': 0, 'previous': None, 'total_count': 2819}


2819

In [5]:
beneficiare_direct = DataFrame(list(
    map(
        lambda k: {
            "userID":k['metadata']['userID'],
            "username":k['metadata']['username'],
            "timeEnd": k['metadata']['timeEnd'],
            "case_id": k['form']['case']['@case_id'],
            "houseold_number_2022": k['form']['household_number'],
            "db_name": k['form']['nom'],
            "db_first_name": k['form']['prenom']
        } 
        ,__objects
    )
))

beneficiare_direct.drop_duplicates('case_id',inplace=True)
beneficiare_direct.reset_index(drop=True)
#beneficiare_direct.case_id.value_counts()

,userID,username,timeEnd,case_id,houseold_number_2022,db_name,db_first_name
0,6b0d5f51d55f4aa4807a736f52304915,2roselord,2022-02-23T15:52:35.368000Z,77de93e3-2858-4535-8056-ba36f5055d78,0,Previlon,Harry Denura
1,6b0d5f51d55f4aa4807a736f52304915,2roselord,2022-02-23T15:51:54.511000Z,a7feeb3e-b9b6-419b-a8ca-8e3086134b11,0,Pasteur,Alens
2,6b0d5f51d55f4aa4807a736f52304915,2roselord,2022-02-23T15:51:13.074000Z,a3810793-b829-4bd3-b4f4-8fd30bbba789,1,Jean Robert,Roselva
3,6b0d5f51d55f4aa4807a736f52304915,2roselord,2022-02-23T15:50:05.234000Z,20a882c0-2b5a-4db3-8b6f-8adc84bcfecd,2,Simeon,Josiane
4,6b0d5f51d55f4aa4807a736f52304915,2roselord,2022-02-23T15:48:28.105000Z,c741f025-98fd-4047-874f-9c64afd807ba,0,Cetoute,Paulmise
...,...,...,...,...,...,...,...
1539,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:41:26.240000Z,88881743-96fe-4e4b-b9e8-5bd30490ad8f,0,Diaquot,Widlet
1540,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:34:11.744000Z,7f6fc269-d750-44e1-b4cf-7446a14960a8,0,Darisma,John Evens
1541,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:26:50.145000Z,c2792c70-6b09-4f69-b2d4-69cc4498ed10,0,Guerrelus,Sony
1542,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:02:48.005000Z,724681f8-f8bd-4fb4-a0b4-c20c1c0b82d0,0,Vincent,Lovena


In [6]:
beneficiare_indirect = DataFrame(list(
    map(
        lambda k: {
            "userID": k['user_id'],
            "ib_case_id": k['case_id'],
            "age": k['properties']['age'],
            "sexe": k['properties']['sexe'],
            "arv": k['properties']['arv'],
            "test": k['properties']['test'],
            "ib_case_id": k['case_id'],
            "parent_case_id": k['indices']['parent']['case_id'],
            "relationship": k['indices']['parent']['relationship'],
        } 
        ,__objects_cases
    )
))

beneficiare_indirect


,userID,ib_case_id,age,sexe,arv,test,parent_case_id,relationship
0,436b6c0759934988b5cf76ced1c46bfd,ca6f342b-7499-4ea3-b9ee-39bec46638a6,2,2,non,non,a81e3aa1b7f3470b8c064788445145e3,child
1,02a25cddec634621822ccaa4657f8246,4a475221-a783-418b-9fcc-42c7ca03a7bd,4,2,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
2,02a25cddec634621822ccaa4657f8246,6b409fc6-cb35-411a-bf82-08987e4ef1c2,6,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
3,02a25cddec634621822ccaa4657f8246,39598e5b-1598-4fc5-96c1-ab962fb7d99d,11,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
4,02a25cddec634621822ccaa4657f8246,82d333a5-444c-4f2c-bea4-f06dc1d2162d,16,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
...,...,...,...,...,...,...,...,...
2814,379d9a3050314f1692130a596e5c2cb9,dd609ffb-0f94-42b9-9c20-84f7fc14753c,12,1,non,non,60b08d15-78f6-4586-a93e-17881870bce1,child
2815,379d9a3050314f1692130a596e5c2cb9,93d96a5c-e1fc-4b54-850f-ed0977c748aa,2,2,non,non,60b08d15-78f6-4586-a93e-17881870bce1,child
2816,6b0d5f51d55f4aa4807a736f52304915,70d6152e-757e-4d03-a9e6-75f2a1e38f73,13,2,non,non,20a882c0-2b5a-4db3-8b6f-8adc84bcfecd,child
2817,6b0d5f51d55f4aa4807a736f52304915,cc3f8863-24b3-4054-a3b7-7be135e57d86,5,1,non,non,20a882c0-2b5a-4db3-8b6f-8adc84bcfecd,child
